# BEE 4750 Lab 4: Simulation-Optimization

**Name**: Akshara Chandrabalan

**ID**: ac2837

> **Due Date**
>
> Friday, 11/17/23, 9:00pm

## Setup

The following code should go at the top of most Julia scripts; it will
load the local package environment and install any needed packages. You
will see this often and shouldn’t need to touch it.

In [1]:
import Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `~/Documents/BEE 5750 /lab04-akshara-c`


In [2]:
using Random # for random seeds
using Distributions # statistical distribution interface
using Roots # find zeros of functions
using Metaheuristics # search algorithms
using Plots # plotting

## Overview

In this lab, you will experiment with simulation-optimization with the
shallow lake problem. The goal of this experimentation is to get an
understanding of how to work with simulation-optimization methods and
the impact of some choices involved in using these methods.

Free free to delete some of the illustrative cells and code blocks in
your notebook as you go through and solve the lab problems…this might
help reduce some potential confusion while grading about what your
answer is.

## Introduction

Due to ongoing economic activity, a town emits phosphorous into a
shallow lake (with a concentration of $a_t$), which also receives
non-point source runoff (concentration $y_t$) from the surrounding area.
The concentration of the lake at time $t+1$ is given by
$$X_{t+1} = X_t + a_t + y_t + \frac{X_t^q}{1+X_t^q} - bX_t,$$

where:

| Parameter | Value                                                |
|:---------:|:-----------------------------------------------------|
|   $a_t$   | point-source phosphorous concentration from the town |
|   $y_t$   | non-point-source phosphorous concentration           |
|    $q$    | rate at which phosphorous is recycled from sediment  |
|    $b$    | rate at which phosphorous leaves the lake            |

and $X_0 = 0$, $y_t \sim LogNormal(\log(0.03), 0.25)$, $q=2.5$, and
$b=0.4$.

The goal of the optimization is to maximize the town’s average
phosphorous concentration releases (as a proxy of economic activity):
$\max \sum_{t=1}^T a_t / T$ over a 100-year period. We have decided
(initially) that an acceptable solution is one which will result in the
lake eutrophying no more than 10% of the time.

The non-point source samples can be sampled using the following code
block:

In [5]:
Random.seed!(1)

T = 100 # length of simualtion
n_samples = 1000 # replace with number of samples if you experiment

P_distribution = LogNormal(log(0.03), 0.25)
y = rand(P_distribution, (T, n_samples)) # sample a T x n_samples matrix


100×1000 Matrix{Float64}:
 0.0294753  0.0459864  0.023513   …  0.0259183  0.0260934  0.0284652
 0.034263   0.0222782  0.0459188     0.0288482  0.0480454  0.0531018
 0.0245199  0.0296271  0.0445619     0.0246404  0.0250734  0.0304308
 0.055448   0.0312     0.0228208     0.0298609  0.0428105  0.0256198
 0.0401417  0.024978   0.0458244     0.0228935  0.0286062  0.0238694
 0.0320754  0.021759   0.0471452  …  0.0472771  0.0187508  0.0306753
 0.0464641  0.0416385  0.0246833     0.0382252  0.0288505  0.0226561
 0.0244027  0.0432707  0.0341214     0.0238988  0.0427204  0.0316143
 0.0231156  0.0279197  0.0217747     0.0231772  0.0335662  0.0324465
 0.0276303  0.0305858  0.0440326     0.0289394  0.0312328  0.0173388
 ⋮                                ⋱                        
 0.025665   0.0341366  0.0274747     0.0283546  0.0458031  0.0277959
 0.0405629  0.0421121  0.0252557     0.0450377  0.0284411  0.0206434
 0.0228445  0.0223746  0.0210942     0.0442834  0.0337672  0.0287835
 0.0252604  0.046

We write the lake model as a function:

In [6]:
# lake function model
# inputs:
#   a: vector of point-source releases (to be optimized)
#   y: randomly-sampled non-point sources
#   q: lake phosphorous recycling rate
#   b: phosphorous outflow rate
# 
# returns:
#   series of lake phosphorous concentrations

function lake(a, y, q, b, T)
    X = zeros(T+1, size(y, 2))
    # calculate states
    for t = 1:T
        X[t+1, :] = X[t, :] .+ a[t] .+ y[t, :] .+ (X[t, :].^q./(1 .+ X[t, :].^q)) .- b.*X[t, :]
    end
    return X
end

lake (generic function with 1 method)

However, this isn’t sufficient on its own! `Metaheuristics.jl` (and most
simulation-optimization packages) require the use of a *wrapper*
function, which accepts as inputs both parameters to be optimized (in
this case, point-source releases `a`) and parameters which will be fixed
(the others; see below for how to incorporate these into the syntax) and
returns the required information for the optimization procedure.

`Metaheuristics.jl` wants its optimizing wrapper function to return (in
order):

-   the objective(s) (in this case, the mean of `a`, $\sum_t a_t / T$),
-   a vector of the degrees to which the solution fails to achieve any
    inequality constraints (positive values indicate a larger failure,
    values below zero are considered acceptable)
-   a vector of the degrees to which the solution fails to achieve any
    equality constraints (only values of zero indicate success), which
    in this case is not relevant, so we just return `[0.0]`.

In [7]:
# function producing optimization outputs
# inputs:
#   a: vector of point-source releases (to be optimized)
#   y: randomly-sampled non-point sources
#   q: lake phosphorous recycling rate
#   b: phosphorous outflow rate
# 
# returns:
#   - objective: mean value of point-source releases
#   - inequality constraint failure vector
#   - equality constraint failure vector (in this case, always [0.0])

function lake_opt(a, y, q, b, T, Xcrit)
    X = lake(a, y, q, b, T)
    # calculate exceedance of critical value
    Pexceed = sum(X[T+1, :] .> Xcrit) / size(X, 2) # proportion of values in the last row of X that exceed the critical value
    failconst = [Pexceed - 0.1] # replace 0.1 if you experiment with the failure probability
    return mean(a), failconst, [0.0]
end


lake_opt (generic function with 1 method)

To optimize using DE (differential evolution), use the following syntax:

``` julia
results = optimize(f, bounds, DE(options=Options(f_calls_limit=max_evals)))
```

where `bounds` is a `Matrix` of lower bounds (first row) and upper
bounds (last row), and `max_evals` is an integer for the maximum number
of evaluations.

-   For example, to set bounds for all decision variables between 0 and
    0.5, you can use

``` julia
bounds = [zeros(T) 0.5ones(T)]'
```

-   Increasing `max_evals` can help you find a better solution, but at a
    larger computational expense.
-   You can use an anonymous function to fix values for non-optimized
    parameters, *e.g.*

``` julia
y = ...
q = ...
b = ...
T = ...
Xcrit = ...
results = optimize(a -> lake_opt(a, y, q, b, t, Xcrit), bounds, DE(options=Options(f_calls_limit=max_evals)))
```

Then to get the approximated minimum value:

``` julia
fx = minimum(result)
```

and the approximated minimizing value:

``` julia
x = minimizer(result)
```

The last piece is to get the critical value (to identify failures),
which we can do using `Roots.jl`, which finds zeros of functions:

In [15]:
q = 2.5
b = 0.4

# define a function whose zeros are the critical values
P_flux(x) = (x^q/(1+x^q)) - b*x # P flux as a function of P concentration (x) 

# use Roots.find_zero() to find the non-eutrophication and non-zero critical value; 
# we know from visual inspection in class that this is bounded between 0.1 and 1.5.
Xcrit = find_zero(P_flux, (0.1, 1.5)); # Xcrit is the threshold 


## Problems

### Problem 1 (2 points)

Using the default setup above, find the approximate optimum value. What
is the value of the objective function, and how many failures (you can
evaluate the `lake` function using your solution to find how many
end-values are above the critical value).

In [39]:
# default setup
max_evals = 1000
bounds = [zeros(T) 0.5ones(T)]' # using default values given above 

# non-optimized parameters 
q = 2.5
b = 0.4
T = 100
n_samples = 1000
y = rand(P_distribution, (T, n_samples)) 

Xcrit = find_zero(P_flux, (0.1, 1.5));

# calling optimizer - objective to maximize a 
results = optimize(a -> lake_opt(a, y, q, b, T, Xcrit), bounds, 
    DE(options=Options(f_calls_limit=max_evals)))


Optimization Result
===================
  Iteration:       1
  Minimum:         0.198871
  Minimizer:       [0.120495, 0.162949, 0.218994, …, 0.292089]
  Function calls:  1000
  Feasibles:       0 / 1000 in final population
  Total time:      13.0924 s
  Stop reason:     Maximum objective function calls exceeded.

In [41]:
# default setup but w/ bounds from 0-0.09
max_evals = 1000
bounds = [zeros(T) 0.09ones(T)]' 

# non-optimized parameters 
q = 2.5
b = 0.4
T = 100
n_samples = 1000
y = rand(P_distribution, (T, n_samples)) 

Xcrit = find_zero(P_flux, (0.1, 1.5));

# calling optimizer - objective to maximize a 
results = optimize(a -> lake_opt(a, y, q, b, T, Xcrit), bounds, 
    DE(options=Options(f_calls_limit=max_evals)))

Optimization Result
===================
  Iteration:       1
  Minimum:         0.0362739
  Minimizer:       [0.0437603, 0.00342541, 0.0542173, …, 0.0858144]
  Function calls:  1000
  Feasibles:       14 / 1000 in final population
  Total time:      13.4574 s
  Stop reason:     Maximum objective function calls exceeded.

In [42]:
# evaluating `lake` function using the solution obtained above to find how many 
# end-values are above the critical value
a = minimizer(results)
Xt = lake(a, y, q, b, T)

fails = sum(Xt[T+1, :] .> Xcrit)
@show fails;


fails = 0


ANSWER: Using the default setup and 1000 function calls, a feasible solution was not found. At the end of the optimization process for this run, the algorithm developed 1000 solutions, with an $a$ value of about 0.198, but none of the solutions resulted in a case where the lake was eutrophied less than 10% of the time (i.e. inequality constraints not satified). So, I ran the optimization again, but this time I restricted the value of $a$ between 0 and 0.09 (i.e. lowering the maximum possible amount of point source P that can enter the lake). Now, given this setup, the optimization algorithm finds 14 feasible solutions, where the approximate optimum value of point source P release (i.e. value of the objective function) is about 0.036. With this setup, there are 0 failures (i.e. inequality constraints were satisfied for all end-values).    

### Problem 2 (5 points)

Feel free to experiment with some of the degrees of freedom in finding
the optimum value. For example:

-   What failure probability are you using to characterize acceptable
    solutions?
-   How many Monte Carlo samples are you using?
-   What bounds are you searching over for the releases?
-   How many function evaluations are you using for the search?
-   What is the impact of different [`Metaheuristics.jl`
    algorithms](https://docs.juliahub.com/Metaheuristics/aJ70z/3.2.12/algorithms/)?

Note that you might want to modify some of these together: for example,
lower acceptable failure probabilities often require more function
evaluations to find acceptable values, and more Monte Carlo samples
increase computational expense, so fewer function evaluations may be
completed in the same time.

Provide a description of what you’ve modified and why. What was the new
solution that you found? Does it satisfy the constraints?

In [47]:
# default setup but with (1) bounds from 0-0.09 and
# (2) higher failure probability of 30%
function lake_opt(a, y, q, b, T, Xcrit)
    X = lake(a, y, q, b, T)
    Pexceed = sum(X[T+1, :] .> Xcrit) / size(X, 2) 
    failconst = [Pexceed - 0.3] # experimenting with failure probability = 0.3
    return mean(a), failconst, [0.0]
end

results = optimize(a -> lake_opt(a, y, q, b, T, Xcrit), bounds, 
    DE(options=Options(f_calls_limit=max_evals)))

Optimization Result
===================
  Iteration:       1
  Minimum:         0.0358672
  Minimizer:       [0.0173959, 0.0405734, 0.0462566, …, 0.021412]
  Function calls:  1000
  Feasibles:       30 / 1000 in final population
  Total time:      13.9826 s
  Stop reason:     Maximum objective function calls exceeded.

In [48]:
a = minimizer(results)
Xt = lake(a, y, q, b, T)

fails = sum(Xt[T+1, :] .> Xcrit)
@show fails;

fails = 0


In [67]:
# default setup but with (1) bounds from 0-0.09 and
# (2) lower failure probability of 5% with higher function evaluations 
max_evals = 2000

function lake_opt(a, y, q, b, T, Xcrit)
    X = lake(a, y, q, b, T)
    Pexceed = sum(X[T+1, :] .> Xcrit) / size(X, 2) 
    failconst = [Pexceed - 0.05] # experimenting with failure probability = 0.05
    return mean(a), failconst, [0.0]
end

results = optimize(a -> lake_opt(a, y, q, b, T, Xcrit), bounds, 
    DE(options=Options(f_calls_limit=max_evals)))


Optimization Result
===================
  Iteration:       2
  Minimum:         0.0316458
  Minimizer:       [0.0257048, 0.0102361, 0.0439606, …, 0.0512995]
  Function calls:  2000
  Feasibles:       480 / 1000 in final population
  Total time:      33.3373 s
  Stop reason:     Maximum objective function calls exceeded.

In [68]:
a = minimizer(results)
Xt = lake(a, y, q, b, T)

fails = sum(Xt[T+1, :] .> Xcrit)
@show fails;

fails = 0


In [69]:
# default setup but with (1) bounds from 0-0.09 and
# (2) doubling the number of Monte Carlo samples while reducing function evaluations
max_evals = 800
n_samples = 2000

function lake_opt(a, y, q, b, T, Xcrit)
    X = lake(a, y, q, b, T)
    Pexceed = sum(X[T+1, :] .> Xcrit) / size(X, 2) 
    failconst = [Pexceed - 0.1] # back to failure probability = 0.1
    return mean(a), failconst, [0.0]
end

results = optimize(a -> lake_opt(a, y, q, b, T, Xcrit), bounds, 
    DE(options=Options(f_calls_limit=max_evals)))


Optimization Result
===================
  Iteration:       1
  Minimum:         0.0322124
  Minimizer:       [0.0577457, 0.0261258, 0.00306537, …, 0.0636815]
  Function calls:  1000
  Feasibles:       349 / 1000 in final population
  Total time:      13.3217 s
  Stop reason:     Maximum objective function calls exceeded.

In [70]:
a = minimizer(results)
Xt = lake(a, y, q, b, T)

fails = sum(Xt[T+1, :] .> Xcrit)
@show fails;

fails = 0


In [64]:
# default setup but now changing bounds
bounds = [zeros(T) 0.08ones(T)]' # also tried upper bound of 0.1  

# back to default values 
max_evals = 1000
n_samples = 1000

function lake_opt(a, y, q, b, T, Xcrit)
    X = lake(a, y, q, b, T)
    Pexceed = sum(X[T+1, :] .> Xcrit) / size(X, 2) 
    failconst = [Pexceed - 0.1] # back to failure probability = 0.1
    return mean(a), failconst, [0.0]
end

results = optimize(a -> lake_opt(a, y, q, b, T, Xcrit), bounds, 
    DE(options=Options(f_calls_limit=max_evals)))

Optimization Result
===================
  Iteration:       1
  Minimum:         0.0333368
  Minimizer:       [0.027823, 0.0105407, 0.0572911, …, 0.0790499]
  Function calls:  1000
  Feasibles:       347 / 1000 in final population
  Total time:      11.0884 s
  Stop reason:     Maximum objective function calls exceeded.

In [66]:
a = minimizer(results)
Xt = lake(a, y, q, b, T)

fails = sum(Xt[T+1, :] .> Xcrit)
@show fails;

fails = 0


ANSWER: I first started with the same setup as in Problem 1, using bounds from 0-0.09, but I experiemnted with a higher failure probability (30%) and a lower failure probability (5%) with 2000 function evaluations (to compenstate for narrower acceptable solution space). In both cases, I got an approximate optimum value of around 0.036 and 0 failures, which is the same result as before. Next, I went back to the default setup, but this time increased the number of Monet Carlo samples by a factor of 2 (reduced the number of function evaluations accordingly for computation time purposes) to see if a wider range of input would result in more feasible solutions and it did, but the approximate optimum value and number of failures is still very similar to before. Lastly, I experimented with changing the bounds. When I increased the upper bound to 0.1, I got 2 feasible solutions, an approximate optimum value of about 0.038, but 32 failures. When I restrict the upper bound to 0.08, I get 347 feasible solutions, an approximate optimum value of about 0.033, but 0 failures. So, there are a few insights we can take from this. First, changing the bounds changed the results most significantly compared to the other parameters I experimented with. Second, when I increased the upper bound, I got a larger approximate optimum value for P release, but there were 32 failures where the inequality constraints were not satisfied. Comparing the results from using an upper bound of 0.09 (from Problem 1), with an upper bound of 0.08, both cases result in 0 failures, but the former setup has an approximate optimum value about 0.036 whereas the latter has an approximate optimum value of about 0.033. This makes sense because when we restrict the bounds, we restrict the range of values from which the algorithm can search for a solution. So, as the upper bound decreases, we get more feasible solutions and 0 failures because we're already forcing P release to be low, which results in the approximate optimum value being lower than it could be if the bounds were relaxed while still maintaining 0 failures. Since we want to maximize P release, because it is a proxy for the town's economic activity, the solution from using an upper bound of 0.09 is preferred becuase it still results in 0 failures while having a larger optimum value.

### Problem 3 (3 points)

What did you learn about the use of these methods? Compare with your
experience with linear programming from earlier in the semester.

ANSWER: With these methods, the solutions are highly variable depending on how you define the problem and select parameter values. For example, with respect to the difference in results obtained from changing the bounds by 0.01, we get notably different solutions that satisfy the problem constraints, but you need to apply context to understand which solutions are best suited for the problem. As per the quote we discussed in class, "The optimal solution of a model is not an optimal solution of a problem unless the model is a perfect representation of the problem", so it is important to formulate the model to fit the problem, and not just to get a solution. Also, increasing the number of evaluations did lead to more feasible solutions, but you have to take into account the computing time when increasing the complexity of the model because you may get a solution, but you have to consider if it's worth the cost. Compared to LPs, simulation-optimization approaches provide more flexibility in the kinds of problems you can solve for becuase you don't have to satisfy the strict linearity requirements that come with using LPs. For problems that are non-linear and more complex, the simulation-optimization approaches would be more suited. However, LPs are more straightforward to use and understand as they are more explicit, whereas the computation and assumptions behind simulation-optimization approaches are not as transparent. Also, LPs provide a more definitive analytic soltuion that is scalable, whereas there is more ambiguity with the simulation-optimization methods because you can't always guarantee a global maximum/minimum.      

## References

Put any consulted sources here, including classmates you worked with/who
helped you.

Lectures 5, 15 and 23 slides.